In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/samsum-dataset-text-summarization/samsum-train.csv
/kaggle/input/samsum-dataset-text-summarization/samsum-test.csv
/kaggle/input/samsum-dataset-text-summarization/samsum-validation.csv
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/dataset_dict.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/validation/state.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/validation/dataset_info.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/validation/data-00000-of-00001.arrow
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/test/state.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/test/dataset_info.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/test/data-00000-of-00001.arrow
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/train/state.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/train/dataset_info.json
/kaggle/input/sam

In [4]:
!pip install datasets


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import pandas as pd

# Loading The Dataset

In [6]:
train_data = pd.read_csv("/kaggle/input/samsum-dataset-text-summarization/samsum-train.csv")
test_data = pd.read_csv("/kaggle/input/samsum-dataset-text-summarization/samsum-test.csv")
val_data = pd.read_csv("/kaggle/input/samsum-dataset-text-summarization/samsum-validation.csv")


train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)
val_data = Dataset.from_pandas(val_data)

# Loading the Model

In [7]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

def preprocess_data(examples):
    """Tokenize the inputs and outputs for training."""
    model_inputs = tokenizer(
        examples['dialogue'], 
        max_length=512, 
        truncation=True, 
        padding="max_length"  
    )
    labels = tokenizer(
        examples['summary'], 
        max_length=128, 
        truncation=True, 
        padding="max_length" 
    )
    
    
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_ids] 
        for label_ids in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Data Preprocessing

In [10]:
def preprocess_data(examples):
    
    
    dialogues = [str(x) for x in examples['dialogue']]
    summaries = [str(x) for x in examples['summary']]
    
    
    model_inputs = tokenizer(
        dialogues,  
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    
   
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            summaries,  
            max_length=128,
            truncation=True,
            padding="max_length"
        )
    
    
    model_inputs["labels"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_ids]
        for label_ids in labels["input_ids"]
    ]
    
    return model_inputs


try:
    train_data = train_data.map(preprocess_data, batched=True, remove_columns=["dialogue", "summary"])
    test_data = test_data.map(preprocess_data, batched=True, remove_columns=["dialogue", "summary"])
    val_data = val_data.map(preprocess_data, batched=True, remove_columns=["dialogue", "summary"])
except Exception as e:
    print(f"Error during preprocessing: {e}")
    raise


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

# Training

In [11]:

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb", 
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,  
    tokenizer=tokenizer,
)

/tmp/ipykernel_23/3213395599.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,1.357400,1.362274
2,1.042500,1.394127
3,0.810100,1.478627
4,0.637900,1.588340
5,0.483200,1.687714


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked t

TrainOutput(global_step=9210, training_loss=0.8925029466258327, metrics={'train_runtime': 13659.9796, 'train_samples_per_second': 5.392, 'train_steps_per_second': 0.674, 'total_flos': 7.981446249578496e+16, 'train_loss': 0.8925029466258327, 'epoch': 5.0})

# Saving the model

In [16]:
trainer.save_model("./fine_tuned_bart")
tokenizer.save_pretrained("./fine_tuned_bart")

('./fine_tuned_bart/tokenizer_config.json',
 './fine_tuned_bart/special_tokens_map.json',
 './fine_tuned_bart/vocab.json',
 './fine_tuned_bart/merges.txt',
 './fine_tuned_bart/added_tokens.json',
 './fine_tuned_bart/tokenizer.json')

# Testing

In [33]:
def summarize_text(text):
    
    device = "cuda" if torch.cuda.is_available() else "cpu" 
    model.to(device)

    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: value.to(device) for key, value in inputs.items()} 

    outputs = model.generate(
        **inputs,
        max_length=50, 
        num_beams=6,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [28]:
import torch


In [34]:
test_dialogue = """ Artificial intelligence (AI) has become a transformative force across various industries, enabling advancements in healthcare, finance, and education. By analyzing large datasets, AI can uncover patterns and provide insights that were previously unattainable. For instance, in healthcare, AI-powered tools can assist doctors in diagnosing diseases early, potentially saving lives. Similarly, in finance, AI algorithms help detect fraudulent transactions and optimize investment strategies. While AI offers immense benefits, ethical concerns, such as data privacy and bias, must be addressed to ensure its responsible use."""

print("Summary:", summarize_text(test_dialogue))


Summary: Artificial intelligence (AI) has become a transformative force across various industries, enabling advancements in healthcare, finance, and education. Ethical concerns must be addressed to ensure its responsible use, such as data privacy and bias. 
